In [ ]:
#r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Unable to open database: 
primary path: d:\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #0 at 'd:\bosss_db_Natconv': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST
No alternative paths specified.
IOException caught while opening database #1 at '\\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #2 at '\\hpccluster\hpccluste

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedBackwardFacingStep");

Project name is set to 'HeatedBackwardFacingStep'.
Opening existing database 'S:\work\scratch\jg11bano\HeatedBackwardFacingStep'.


In [ ]:
var myBatch = (SlurmClient)GetDefaultQueue();
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+2000});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands

index,value
0,#SBATCH -C avx512
1,#SBATCH --mem-per-cpu=2000


In [ ]:
static var myDb = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();


In [ ]:
myDb

{ Session Count = 0; Grid Count = 0; Path = S:\work\scratch\jg11bano\HeatedBackwardFacingStep }

In [ ]:
// static var myDb = myBatch.CreateOrOpenCompatibleDatabase("HeatedBackwardFacingStep");

// static var myBatch = (SlurmClient)BoSSSshell.ExecutionQueues[1];
// static var myDb = BoSSS.Application.BoSSSpad.BoSSSshell.OpenOrCreateDatabase(@"S:\work\scratch\jg11bano\Heated_BFS"); 




In [ ]:
int Resolution =16;
int dg = 2;
// double[] ReynoldsS = new double[]{100, 400,700,1000};
// double[] ExpansionRatioS = new double[]{1.5,2.0,2.5};  // Defines the length of the inlet(h). ER = 1+S/h;
double[] ReynoldsS = new double[]{700};
double[] ExpansionRatioS = new double[]{2.0};
int MPI_PROCS = 16;

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int Res, double S, double h, double H, double L0,double L) {
        // Grid2D grd;

        double[] CutOut1Point1 = new double[2] { 0.0, 0.0 };
        double[] CutOut1Point2 = new double[2] { -L0, S };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        
        // left part
        var _leftNodes = GenericBlas.SinLinSpacing(-2 * L0, 0, 0.98, (2 * Res) + 1);
        var leftNodes = _leftNodes.GetSubVector(_leftNodes.Length / 2, _leftNodes.Length / 2 + 1);
        //right part
        int xNodesMultiplier  = 110;
        var _rightNodes = GenericBlas.SinLinSpacing(0, 2 * L, 0.9, (xNodesMultiplier * Res) + 1);
        var rightNodes = _rightNodes.GetSubVector(0, _rightNodes.Length / 2 + 1);

        List<double> listX = new List<double>();
        listX.AddRange(leftNodes.Take(leftNodes.Count() - 1).ToList());
        listX.AddRange(rightNodes.Take(rightNodes.Count() + 1).ToList());
        var xNodes = listX.ToArray();


        //Bottom part
        var _bottomNodes = GenericBlas.SinLinSpacing(0, S, 0.8, 2*(2 * Res) + 1);
        //upper part
        var _upperNodes = GenericBlas.SinLinSpacing(S, S+h, 0.8, 2*(2 * Res) + 1);
        // var upperNodes = _upperNodes.GetSubVector(0, _upperNodes.Length / 2 + 1);        
        // for(int i = 0; i < upperNodes.Count(); i++){
        //     upperNodes[i] = upperNodes[i] + S;
        // }
        
        List<double> listY = new List<double>();
        listY.AddRange(_bottomNodes.Take(_bottomNodes.Count() - 1).ToList());
        listY.AddRange(_upperNodes.Take(_upperNodes.Count() + 1).ToList());
        var yNodes = listY.ToArray();
        var grd = Grid2D.Cartesian2DGrid(xNodes, yNodes, CutOuts: CutOut1);

        Console.WriteLine("Number of nodes in X direction:" +listX.Count() );
        Console.WriteLine("Number of nodes in Y direction:" +yNodes.Count() );
        grd.EdgeTagNames.Add(1, "Velocity_Inlet");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        grd.EdgeTagNames.Add(2, "NoSlipNeumann");
        grd.EdgeTagNames.Add(4, "Wall");

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Inlet oxidizer
            if (Math.Abs(x + L0) < 1e-8)
                return 1;

            //Outlet
            if (Math.Abs(x - L) < 1e-8)
                return 3;

            //Bottom Wall
            if (Math.Abs(y - 0) < 1e-8 && x > 1e-8)
                return 4;

            return 2; 
        });

        myDb.SaveGrid(ref grd);

        return grd;
    }
}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double const_val, double S, double h, double umean ) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");


           stw.WriteLine("  static public double ConstantVal(double[] X) {");
           stw.WriteLine("    return " + const_val+";");
           stw.WriteLine("  }");

           double H = S+h;           
           stw.WriteLine("  static public double ParabolaVelocity(double[] X) {");
           stw.WriteLine("    return  -6 * "+umean+" * (X[1] -"+ S+") * (X[1] - 1 * "+H+") / ("+h * h+") ;");
           stw.WriteLine("  }");


           stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
    static public Formula Get_ConstantVal(double const_val, double S, double h,double umean){
        return new Formula("BoundaryValues.ConstantVal", AdditionalPrefixCode:GetPrefixCode(const_val,S,h,umean));
    }

    static public Formula Get_ParabolaVelocity(double const_val, double S, double h, double umean){
        return new Formula("BoundaryValues.ParabolaVelocity", AdditionalPrefixCode:GetPrefixCode(const_val,S,h,umean));
    }

}

## Send and run jobs

In [ ]:
var controls = new List<XNSEC_Control>();

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int Resolution, double _Reynolds, bool HeatOn, double expansionRatio) {
    XNSEC_Control C = new XNSEC_Control();
    Console.WriteLine("////////////////////////////////////////////////////////////////////////////////////");
    Console.WriteLine("BackwardFacingStep2");
    Console.WriteLine("////////////////////////////////////////////////////////////////////////////////////");

    // Solver configuration
    // ==============

    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

    C.ImmediatePlotPeriod = 1;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.verbose = true;
    C.ProjectName = "Heated_BackwardFacingStep_Lichtenberg_2";
    int enOK = HeatOn == true ? 1 : 0;
    C.SessionName = C.ProjectName + "dg" + dg + "k" + Resolution + "Re" + _Reynolds + "EnergyOK" + enOK;
    //C.physicsMode = PhysicsMode.LowMach;
    C.physicsMode = PhysicsMode.Combustion;
    C.EnableMassFractions = false;
    C.EnableTemperature = false;
    C.MatParamsMode = MaterialParamsMode.Constant;

    C.rhoOne = true;
    if (HeatOn) {
        C.MatParamsMode = MaterialParamsMode.Sutherland;
        C.rhoOne = false;
        C.EnableTemperature = true;
    }
    C.AnalyticsolutionSwitch = false;

    C.PhysicalParameters.IncludeConvection = true;

    C.NumberOfChemicalSpecies = 1;
    C.ChemicalReactionActive = false;
    C.SetDGdegree(dg);
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 };

    C.HeatRelease = 0.0;
    // Parameters
    // ==============

    C.Reynolds = _Reynolds;
    C.Prandtl = 0.7132;
    C.Schmidt = 1.0;
    C.PenaltyViscMomentum = 1.0;
    C.PenaltyHeatConduction = 1.0;

    C.NonLinearSolver.verbose = true;
    // C.LinearSolver.verbose = true;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.LinearSolver.NoOfMultigridLevels = 3;
    C.NonLinearSolver.ConvergenceCriterion = 1e-5;
    // C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    // C.NoOfTimesteps = 10000;
    // C.dtFixed = 0.5;
    
    // C.HomotopyApproach = XNSEC_Control.HomotopyType.Automatic;
    // C.HomotopyVariable = XNSEC_Control.HomotopyVariableEnum.Reynolds;
    // C.homotopieAimedValue = Math.Sqrt(C.Reynolds);
    // C.StartingHomotopyValue = 10; // Suficiently easy to find solution
    // C.NonLinearSolver.HomotopyStepLongFail = 20;

    //C.ImmediatePlotPeriod = 1;
    // Grid declaration
    // ===============

     double Sdim = 15;  //mm, dim step height
     double hdim = Sdim/(expansionRatio-1.0);
     double Hdim = hdim+Sdim; // mm channel height


     double LREF = Sdim;// step height, paper from XIE

    // static double LREF = 2*hdim; // Hydraulic referemce, paper from Armaly
     double H = Hdim / LREF;// nondim channel height
     double S = Sdim / LREF;  //nondim step height
     double h = hdim/LREF;
     double L0 = S; // nondim upstream wall
    // static double L = 30 * S; // downstream wall
     double L = 50 * S; // downstream wall

    

    C.SetGrid(GridFactory.GenerateGrid(Resolution, S,h,H,L0,L));  //

    double TRef = 273 + 10;
    double Tin = (273 + 10) / TRef;
    double Twall = (273 + 40) / TRef;

    // initial values
    // ==============

    double dummy = 0;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantVal(0.5, dummy, dummy, dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantVal(0.0, dummy, dummy, dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantVal(0.0, dummy, dummy, dummy));
    C.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantVal(Tin, dummy, dummy, dummy));
    C.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantVal(1.0, dummy, dummy, dummy));

    // boundary conditions
    // ===================

    C.AddBoundaryValue("Wall", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantVal(Twall, dummy, dummy, dummy));
    C.AddBoundaryValue("Wall", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantVal(1.0, dummy, dummy, dummy));



    double velMean= 1.0;
    C.AddBoundaryValue("Velocity_Inlet", VariableNames.Velocity_d(0) + "#A", BoundaryValueFactory.Get_ParabolaVelocity(-11111.0, S,h, velMean));
    C.AddBoundaryValue("Velocity_Inlet", VariableNames.Velocity_d(1) + "#A", BoundaryValueFactory.Get_ConstantVal(0.0, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet", VariableNames.Temperature + "#A", BoundaryValueFactory.Get_ConstantVal(Tin, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet", VariableNames.MassFraction0 + "#A", BoundaryValueFactory.Get_ConstantVal(1.0, dummy, dummy, dummy));

    C.AddBoundaryValue("Pressure_Outlet");
    return C;
}

## Run Simulations

In [ ]:
foreach(int Re in ReynoldsS){
    foreach(int ER in ExpansionRatioS){
        Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
        var C = GiveMeTheCtrlFile(dg, Resolution, Re, true, ER); 
        string jobName       = C.SessionName;
        Console.WriteLine(jobName);
        var oneJob           = new Job(jobName, solver);
        oneJob.SetControlObject(C);
        myBatch.ExecutionTime            = "0:59:00";
        oneJob.NumberOfMPIProcs = MPI_PROCS;
        List<string> Cmdtmp = (myBatch as SlurmClient).AdditionalBatchCommands.ToList();
        Cmdtmp.Add($"#SBATCH -N 1");
        (myBatch as SlurmClient).AdditionalBatchCommands = Cmdtmp.ToArray();
        oneJob.Activate(myBatch); 
    }
}

////////////////////////////////////////////////////////////////////////////////////
BackwardFacingStep2
////////////////////////////////////////////////////////////////////////////////////

Number of nodes in X direction:897
Number of nodes in Y direction:129
Grid Edge Tags changed.
Heated_BackwardFacingStep_Lichtenbergdg2k16Re400EnergyOK1
Deploying job Heated_BackwardFacingStep_Lichtenbergdg2k16Re400EnergyOK1 ... 
Deploying executables and additional files ...
Deployment directory: T:\BoSSS\HeatedBackwardFacingStep-XNSEC2022Feb18_165224
copied 56 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
// // // wait for all jobs to finish (up to 1 day, check every 1 minutes)
// BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*1), PollingIntervallSeconds:(60*1));

In [ ]:
// // detect failed Jobs in the job management
// var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
//     .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
//                   || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
// suspects

In [ ]:
// NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

In [ ]:
// 